# Experiment: Fine Tuning Whisper Small

The goal of this experiment is to fine tune the Whisper Small model on the TIL dataset and act as a baseline for future experiments.

- Fine tuning on TIL base training data
- No data augmentation to start with
- Greedy decoding


In [ ]:
# Run this if required
# %pip install transformers evaluate datasets librosa jiwer wandb


I will track progress on Weights and Biases


In [ ]:
!wandb login

In [ ]:
%env WANDB_PROJECT=TIL2023-ASR

#### Imports


In [ ]:
import wandb
import torch
import evaluate

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union

from transformers import (
    WhisperProcessor,
    WhisperFeatureExtractor,
    WhisperTokenizerFast,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from datasets import load_from_disk

#### Load Model


In [ ]:
PRETRAINED_MODEL = "openai/whisper-small"


In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(PRETRAINED_MODEL)
tokenizer = WhisperTokenizerFast.from_pretrained(PRETRAINED_MODEL)
preprocessor = WhisperProcessor.from_pretrained(PRETRAINED_MODEL)
forced_decoder_ids = preprocessor.get_decoder_prompt_ids(
    language="english", task="transcribe"
)

#### Load Data


In [ ]:
TRAIN_PATH = "../../data/train/til_asr_base_train/"


In [ ]:
train_dataset = load_from_disk(TRAIN_PATH)
print(train_dataset)


In [ ]:
# Look at the first sample
print(train_dataset["train"][0])


#### Prepare Data


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
train_dataset = train_dataset.map(prepare_dataset)


In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"]}
            for feature in features
        ]
        batch = self.processor.feature_extractor.pad(
            input_features, return_tensors="pt"
        )

        # get the tokenized label sequences
        label_features = [
            {"input_ids": feature["labels"]} for feature in features
        ]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(
            label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (
            (labels[:, 0] == self.processor.tokenizer.bos_token_id)
            .all()
            .cpu()
            .item()
        ):
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(preprocessor)


#### Train Model


In [ ]:
wer_metric = evaluate.load("wer")


In [ ]:
def normalize_to_til(transcript: str) -> str:
    # TIL output is purely uppercase alphabet and space
    # so we normalize the output to that
    result = "".join([c.upper() if c.isalpha() else " " for c in transcript])
    # Remove double spaces
    while "  " in result:
        result = result.replace("  ", " ")
    return result

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # normalize to TIL output
    pred_str = [normalize_to_til(transcript) for transcript in pred_str]
    label_str = [normalize_to_til(transcript) for transcript in label_str]

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
# Load the model
model = WhisperForConditionalGeneration.from_pretrained(PRETRAINED_MODEL)
model.config.forced_decoder_ids = forced_decoder_ids


In [ ]:
# Define Training Arguments
OUTPUT_DIR = "whisper-small-til-baseline-ft-1"
GENERATION_MAX_LENGTH = 100
SAVE_STEPS = 1000
EVAL_STEPS = 1000
REPORT_TO = ["wandb", "tensorboard"]

##### Training

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=SAVE_STEPS,
    eval_steps=EVAL_STEPS,
    logging_steps=25,
    report_to=REPORT_TO,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=train_dataset["train"],
    eval_dataset=train_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=preprocessor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "language": "English",
    "model_name": "FloralArrangements",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

trainer.push_to_hub(**kwargs)